In [1]:
!make -C code clean

import os, platform
import numpy as np
#np.show_config()

make -C 01_mul clean
rm -rf *.o *.dSYM/ mul
make -C 02_fma clean
rm -rf *.o *.dSYM/ fma
make -C 03_omp clean
rm -rf *.o *.dSYM/ omp


# SIMD (vector processing)

1. Types of parallelism
   1. Shared-memory parallelism
   2. Distributed-memory parallelism
   3. Vector processing
2. SIMD instructions
   1. CPU capabilities
   2. x86 intrinsic functions
   3. Symbol table
   4. Inspect assembly: 1, 3, 5 multiplications

# Types of parallelism

The popular computer architecture is based on sequential processing.  The most fundamental processing unit executes instructions one by one.

<center><img src="image/architecture.png" alt="Common computer architecture" /></center>

If we assume the processor can only perform sequential processing, we need to use multiple processors to perform parallel processing.  Differentiated by the memory access, the parallelism can be broadly set to two categories:

* Shared-memory parallel processing
* Distributed-memory parallel processing

## Shared-memory parallel processing

<br />
<center><img src="image/shared.png" alt="Shared-memory parallelism" /></center>

## Distributed-memory parallel processing

<br />
<center><img src="image/distributed.png" alt="Distributed-memory parallelism" /></center>

# Vector processing

When the parallelism happens in the processor (one processing unit or core), it is usually done once for a single instruction with multiple data (SIMD).  It has also been called vector processing (it's an illustrative name).

Before showing what is vector processing, let us see the ordinary scalar execution:

<center><img src="image/scalar.png" alt="Scalar execution" /></center>

The vector execution uses a wider register so that it can perform an operation for multiple data at once:

<center><img src="image/vector.png" alt="Vector execution" /></center>

# Check CPU capabilities

To take advantage of SIMD, we will need to inspect the CPU instructions, or the assembly.  But most of the time we stay in high-level languages.  The way we deal with the assembly is to get familiar with the instructions, e.g., using [x86 and amd64 instruction reference](https://www.felixcloutier.com/x86/).

x86 provides a series of SIMD instructions, including

* 64-bit: MMX
* 128-bit: SSE, SSE2, SSE3, SSE4, SSE4.1, SSE4.2 (streaming simd extension)
* 256-bit: AVX, AVX2 (advanced vector extension)
* 512-bit: AVX-512

Recent processors usually are equipped with AVX2, which was released with Haswell in 2013.  Before asking the compiler to use the specific instruction set, query the operating system for the cpu capabilities.

In [2]:
print("Check on", platform.system())
if 'Linux' == platform.system():
    # check whether your cpu supports avx2 on linux
    !grep flags /proc/cpuinfo
elif 'Darwin' == platform.system():
    # check whether your cpu supports avx2 on mac
    !sysctl -a | grep machdep.cpu.*features

Check on Darwin
machdep.cpu.features: FPU VME DE PSE TSC MSR PAE MCE CX8 APIC SEP MTRR PGE MCA CMOV PAT PSE36 CLFSH DS ACPI MMX FXSR SSE SSE2 SS HTT TM PBE SSE3 PCLMULQDQ DTES64 MON DSCPL VMX EST TM2 SSSE3 FMA CX16 TPR PDCM SSE4.1 SSE4.2 x2APIC MOVBE POPCNT AES PCID XSAVE OSXSAVE SEGLIM64 TSCTMR AVX1.0 RDRAND F16C
machdep.cpu.leaf7_features: RDWRFSGS TSC_THREAD_OFFSET SGX BMI1 AVX2 SMEP BMI2 ERMS INVPCID FPU_CSDS MPX RDSEED ADX SMAP CLFSOPT IPT MDCLEAR TSXFA IBRS STIBP L1DF SSBD
machdep.cpu.extfeatures: SYSCALL XD 1GBPAGE EM64T LAHF LZCNT PREFETCHW RDTSCP TSCI


# x86 intrinsic functions

Major compilers provide header files for using the intrinsic functions that can be directly translated into the SIMD instructions:

* `<mmintrin.h>`: MMX
* `<xmmintrin.h>`: SSE
* `<emmintrin.h>`: SSE2
* `<pmmintrin.h>`: SSE3
* `<tmmintrin.h>`: SSSE3
* `<smmintrin.h>`: SSE4.1
* `<nmmintrin.h>`: SSE4.2
* `<ammintrin.h>`: SSE4A
* `<immintrin.h>`: AVX
* `<zmmintrin.h>`: AVX512

You may also use `<x86intrin.h>` which includes everything.

With the intrinsic functions, programmers don't need to really write assembly, and can stay in the high-level languages most of the time.  The intrinsic functions correspond to x86 instructions.  An example of using it:

```cpp
__m256 * ma = (__m256 *) (&a[i*width]);
__m256 * mb = (__m256 *) (&b[i*width]);
__m256 * mr = (__m256 *) (&r[i*width]);
*mr = _mm256_mul_ps(*ma, *mb);
```

**Intel intrinsic guide**: Intel maintains a website to show the available intrinsics: https://software.intel.com/sites/landingpage/IntrinsicsGuide/ .  Consult and remember it when needed.

Using intrinsics and SIMD for optimization is a tedious process.  The materials presented here are not a complete guide to you, but show you one way to study and measure the benefits.  The measurement is important to assess whether or not you need the optimization.

We will use the example, `01_mul/mul.cpp`, to show how to use the 256-bit-wide AVX to perform vector multiplication for 8 single-precision floating-point values.

```cpp
constexpr const size_t width = 8;
constexpr const size_t repeat = 1024 * 1024;
constexpr const size_t nelem = width * repeat;
```

In [3]:
# time the difference between the loop and the simd/avx versions
!make -C code/01_mul run

g++  -std=c++17 -g -O3 -m64 -mavx  -c -o mul.o mul.cpp
g++  -std=c++17 -g -O3 -m64 -mavx   -o mul mul.o
./mul
width: 8
nelem: 8388608

arr: 0x0x7fbf40800000
brr: 0x0x7fbf42800000
rrr1: 0x0x7fbf44800000
rrr2: 0x0x7fbf46800000

Timing repeats for 20 times and takes the minimum

1 multiplication by loop takes: 0.00507986 sec
1 multiplication by simd takes: 0.00350077 sec

3 multiplication by loop takes: 0.0111593 sec
3 multiplication by simd takes: 0.00343282 sec

5 multiplication by loop takes: 0.0219515 sec
5 multiplication by simd takes: 0.00387762 sec



## Symbol table

I use [radare2](https://rada.re/n/) to inspect the assembly of the generated image.  Before really checking the assembly, we need to identify what functions to be inspected from the symbol table.

In [4]:
# take a look at the symbol table
!make -C code/01_mul r2sym

r2 -Aqc "e scr.color=0 ; afl" mul
0x100002640   32 3553 -> 3450 entry0
0x100001720    3 178          sym.multiply1_loop_float__float__float_
0x1000017e0    3 102          sym.multiply1_simd_float__float__float_
0x100001850    3 354          sym.multiply3_loop_float__float__float_
0x1000019c0    3 107          sym.multiply3_simd_float__float__float_
0x100001a30    3 546          sym.multiply5_loop_float__float__float_
0x100001c60    3 87           sym.multiply5_simd_float__float__float_
0x100001cc0   58 1723 -> 1641 sym.run_std::__1::function_void__float__float__float____float__float__float_
0x1000023a0    5 638          sym.check_float__float_
0x100003530    6 249          method.std::__1::basic_ostream_char__std::__1::char_traits_char____std::__1::__put_character_sequence_char__std::__1.char_traits_char____std::__1::basic_ostream_char__std::__1::char_traits_char_____char_const__unsigned_long_
0x1000038f6    1 6            sym.imp.std::__1::basic_ostream_char__std::__1::char_traits_cha

## 1 multiplication

To demonstrate the effect of different ratio of calculations to memory access, I use 3 sets of multiplication.  The first set uses 1 multiplication:

```cpp
void multiply1_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * b[j];
        }
    }
}

void multiply1_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *mb);
    }
}
```

In [5]:
# 1 multiplication with loop
!make -C code/01_mul r2 NAME=multiply1_loop

r2 -Aqc "e scr.color=0 ; sf sym.multiply1_loop_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x100002c10
            ;-- func.100001720:
┌ 178: sym.multiply1_loop_float__float__float_ ();
│           0x100001720      55             push rbp                   ; multiply1_loop(float*, float*, float*)
│           0x100001721      4889e5         mov rbp, rsp
│           0x100001724      48c7c0f8ffff.  mov rax, 0xfffffffffffffff8
│           0x10000172b      0f1f440000     nop dword [rax + rax]
│           ; CODE XREF from multiply1_loop(float*, float*, float*) @ 0x1000017ca
│       ┌─> 0x100001730      c5fa10448720   vmovss xmm0, dword [rdi + rax*4 + 0x20]
│       ╎   0x100001736      c5fa59448620   vmulss xmm0, xmm0, dword [rsi + rax*4 + 0x20]
│       ╎   0x10000173c      c5fa11448220   vmovss dword [rdx + rax*4 + 0x20], xmm0
│       ╎   0x100001742      c5fa10448724   vmovss xmm0, dword [rdi + rax*4 + 0x24]
│       ╎   0x100001748      c5fa59448624   vmulss xmm0, 

In [6]:
# 1 multiplication with simd/avx
!make -C code/01_mul r2 NAME=multiply1_simd

r2 -Aqc "e scr.color=0 ; sf sym.multiply1_simd_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x100002d03
            ;-- func.1000017e0:
┌ 102: sym.multiply1_simd_float__float__float_ ();
│           0x1000017e0      55             push rbp                   ; multiply1_simd(float*, float*, float*)
│           0x1000017e1      4889e5         mov rbp, rsp
│           0x1000017e4      31c0           xor eax, eax
│           0x1000017e6      662e0f1f8400.  nop word cs:[rax + rax]
│           ; CODE XREF from multiply1_simd(float*, float*, float*) @ 0x10000183f
│       ┌─> 0x1000017f0      c5fc280407     vmovaps ymm0, ymmword [rdi + rax]
│       ╎   0x1000017f5      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
│       ╎   0x1000017fa      c5fc290402     vmovaps ymmword [rdx + rax], ymm0
│       ╎   0x1000017ff      c5fc28440720   vmovaps ymm0, ymmword [rdi + rax + 0x20]
│       ╎   0x100001805      c5fc59440620   vmulps ymm0, ymm0, ymmword [rsi + rax + 0x20

## 3 multiplication

The second set uses 3 multiplications:

```cpp
void multiply3_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply3_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [7]:
# 3 multiplication with loop
!make -C code/01_mul r2 NAME=multiply3_loop

r2 -Aqc "e scr.color=0 ; sf sym.multiply3_loop_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x100002e68
            ;-- func.100001850:
┌ 354: sym.multiply3_loop_float__float__float_ ();
│           0x100001850      55             push rbp                   ; multiply3_loop(float*, float*, float*)
│           0x100001851      4889e5         mov rbp, rsp
│           0x100001854      48c7c0f8ffff.  mov rax, 0xfffffffffffffff8
│           0x10000185b      0f1f440000     nop dword [rax + rax]
│           ; CODE XREF from multiply3_loop(float*, float*, float*) @ 0x1000019aa
│       ┌─> 0x100001860      c5fa10448720   vmovss xmm0, dword [rdi + rax*4 + 0x20]
│       ╎   0x100001866      c5fa59c0       vmulss xmm0, xmm0, xmm0
│       ╎   0x10000186a      c5fa11448220   vmovss dword [rdx + rax*4 + 0x20], xmm0
│       ╎   0x100001870      c5fa59448620   vmulss xmm0, xmm0, dword [rsi + rax*4 + 0x20]
│       ╎   0x100001876      c5fa11448220   vmovss dword [rdx + rax*4 + 0

In [8]:
# 3 multiplication with simd/avx
!make -C code/01_mul r2 NAME=multiply3_simd

r2 -Aqc "e scr.color=0 ; sf sym.multiply3_simd_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x100002f67
            ;-- func.1000019c0:
┌ 107: sym.multiply3_simd_float__float__float_ ();
│           0x1000019c0      55             push rbp                   ; multiply3_simd(float*, float*, float*)
│           0x1000019c1      4889e5         mov rbp, rsp
│           0x1000019c4      31c0           xor eax, eax
│           0x1000019c6      662e0f1f8400.  nop word cs:[rax + rax]
│           ; CODE XREF from multiply3_simd(float*, float*, float*) @ 0x100001a24
│       ┌─> 0x1000019d0      c5fc280407     vmovaps ymm0, ymmword [rdi + rax]
│       ╎   0x1000019d5      c5fc59c0       vmulps ymm0, ymm0, ymm0
│       ╎   0x1000019d9      c5fc290402     vmovaps ymmword [rdx + rax], ymm0
│       ╎   0x1000019de      c5fc590406     vmulps ymm0, ymm0, ymmword [rsi + rax]
│       ╎   0x1000019e3      c5fc290402     vmovaps ymmword [rdx + rax], ymm0
│       ╎   0x1000019e8    

## 5 multiplication

The third (last) set uses 5 multiplications:

```cpp
void multiply5_loop(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat*width; i+=width)
    {
        for (size_t j=i; j<i+width; ++j)
        {
            r[j] = a[j] * a[j];
            r[j] *= a[j];
            r[j] *= b[j];
            r[j] *= b[j];
            r[j] *= b[j];
        }
    }
}

void multiply5_simd(float* a, float* b, float* r)
{
    for (size_t i=0; i<repeat; ++i)
    {
        __m256 * ma = (__m256 *) (&a[i*width]);
        __m256 * mb = (__m256 *) (&b[i*width]);
        __m256 * mr = (__m256 *) (&r[i*width]);
        *mr = _mm256_mul_ps(*ma, *ma);
        *mr = _mm256_mul_ps(*mr, *ma);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
        *mr = _mm256_mul_ps(*mr, *mb);
    }
}
```

In [9]:
# 5 multiplication with loop
!make -C code/01_mul r2 NAME=multiply5_loop

r2 -Aqc "e scr.color=0 ; sf sym.multiply5_loop_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x1000030d2
            ;-- func.100001a30:
┌ 546: sym.multiply5_loop_float__float__float_ ();
│           0x100001a30      55             push rbp                   ; multiply5_loop(float*, float*, float*)
│           0x100001a31      4889e5         mov rbp, rsp
│           0x100001a34      48c7c0f8ffff.  mov rax, 0xfffffffffffffff8
│           0x100001a3b      0f1f440000     nop dword [rax + rax]
│           ; CODE XREF from multiply5_loop(float*, float*, float*) @ 0x100001c4a
│       ┌─> 0x100001a40      c5fa10448720   vmovss xmm0, dword [rdi + rax*4 + 0x20]
│       ╎   0x100001a46      c5fa59c0       vmulss xmm0, xmm0, xmm0
│       ╎   0x100001a4a      c5fa11448220   vmovss dword [rdx + rax*4 + 0x20], xmm0
│       ╎   0x100001a50      c5fa59448720   vmulss xmm0, xmm0, dword [rdi + rax*4 + 0x20]
│       ╎   0x100001a56      c5fa11448220   vmovss dword [rdx + rax*4 + 0

In [10]:
# 5 multiplication with simd/avx
!make -C code/01_mul r2 NAME=multiply5_simd

r2 -Aqc "e scr.color=0 ; sf sym.multiply5_simd_float__float__float_ ; pdf" mul
            ; DATA XREF from entry0 @ 0x1000031d1
            ;-- func.100001c60:
┌ 87: sym.multiply5_simd_float__float__float_ ();
│           0x100001c60      55             push rbp                   ; multiply5_simd(float*, float*, float*)
│           0x100001c61      4889e5         mov rbp, rsp
│           0x100001c64      31c0           xor eax, eax
│           0x100001c66      662e0f1f8400.  nop word cs:[rax + rax]
│           ; CODE XREF from multiply5_simd(float*, float*, float*) @ 0x100001cb0
│       ┌─> 0x100001c70      c5fc280407     vmovaps ymm0, ymmword [rdi + rax]
│       ╎   0x100001c75      c5fc59c0       vmulps ymm0, ymm0, ymm0
│       ╎   0x100001c79      c5fc290402     vmovaps ymmword [rdx + rax], ymm0
│       ╎   0x100001c7e      c5fc590407     vmulps ymm0, ymm0, ymmword [rdi + rax]
│       ╎   0x100001c83      c5fc290402     vmovaps ymmword [rdx + rax], ymm0
│       ╎   0x100001c88     

# OpenMP

In [11]:
!make -C code/03_omp run

clang++ -Xpreprocessor -fopenmp -std=c++17 -g -O3  -c -o omp.o omp.cpp
clang++ -Xpreprocessor -fopenmp -std=c++17 -g -O3  -lomp -o omp omp.o
./omp
Hello from thread 0, nthreads 8
Hello from thread 4, nthreads 8
Hello from thread 3, nthreads 8
Hello from thread 6, nthreads 8
Hello from thread 2, nthreads 8
Hello from thread 7, nthreads 8
Hello from thread 1, nthreads 8
Hello from thread 5, nthreads 8


In [12]:
!env OMP_NUM_THREADS=1 make -C code/03_omp run

./omp
Hello from thread 0, nthreads 1


In [13]:
!env OMP_NUM_THREADS=5 make -C code/03_omp run

./omp
Hello from thread 0, nthreads 5
Hello from thread 3, nthreads 5
Hello from thread 1, nthreads 5
Hello from thread 2, nthreads 5
Hello from thread 4, nthreads 5


# Exercises

1. Replace the single-precision floating-point vector type `__m256` with the double-precision floating-point vector type `__m256d` in the example, and compare the performance with the sinple-precision version.

# References

1. Crunching Numbers with AVX and AVX2 (AVX tutorials): https://www.codeproject.com/Articles/874396/Crunching-Numbers-with-AVX-and-AVX
2. Agner Fog (Agner's website): https://www.agner.org

   * Instruction table (latency information): https://www.agner.org/optimize/instruction_tables.pdf
   * Software optimization resources: https://www.agner.org/optimize/
3. x86 and amd64 instruction reference (unofficial) by Félix Cloutier: https://www.felixcloutier.com/x86/
4. Intel Intrinsics Guide: https://software.intel.com/sites/landingpage/IntrinsicsGuide/
5. Computer Organization and Assembly Languages by Yung-Yu Chuang, NTU: https://www.csie.ntu.edu.tw/~cyy/courses/assembly/12fall/news/